In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_hdf('etfdata_full.h5', 'etf_df');

In [3]:
data = data.reset_index();
data.dropna();

In [4]:
data = data[data['VOL'] != 0];

In [5]:
data['vol change'] = data['VOL'].astype(float).pct_change();

In [6]:
data.groupby('TICKER').first()['vol change'] = 0;

In [7]:
zscore = lambda x: (x - x.mean())/x.std();

In [8]:
data['vol_change_z'] = data.groupby('TICKER')['vol change'].transform(zscore);
data.dropna();

In [9]:
data['reversal'] = (data['vol_change_z'] > -0.84) & (data['vol_change_z'] < 0.53);

In [10]:
reversal=data.set_index(['TICKER','DATE'])
reversal=reversal.reorder_levels(['DATE','TICKER'])
reversal.sort_index(inplace=True)
reversal = reversal.dropna();

In [11]:
def quintiles(inser):
    outser = pd.qcut(inser, q=5, labels=range(1,6))
    return outser

In [12]:
reversal['QUINTILE'] = reversal['RET'].groupby(['DATE']).apply(quintiles);

In [13]:
reversal_q = reversal.groupby(['QUINTILE','DATE'])['RET'].mean();

In [14]:
reversal['strat'] = 0;
reversal['strat']=reversal['strat'].where((reversal['reversal'] != 1) | (reversal['QUINTILE'] != 1),1);
reversal['strat']=reversal['strat'].where((reversal['reversal'] != 1) | (reversal['QUINTILE'] != 5),-1);

In [15]:
reversal['shift_ret'] = reversal.groupby('TICKER')['RET'].shift(-5);

In [16]:
reversal['strat_ret'] = reversal['shift_ret'] * reversal['strat']

In [17]:
reversal['strat_ret'].describe()

count    3.445017e+06
mean     8.419508e-05
std      1.313953e-02
min     -7.780664e-01
25%      0.000000e+00
50%     -0.000000e+00
75%      0.000000e+00
max      7.405064e-01
Name: strat_ret, dtype: float64

In [18]:
sharpe = reversal['strat_ret'].describe()['mean']/reversal['strat_ret'].describe()['std'] * np.sqrt(250)
sharpe

0.10131571487543953

In [25]:
ret = reversal['strat_ret'].dropna()
max_dd = (ret/ret.cummax()-1).replace(-np.inf, np.nan).min()
max_dd

-5.9259258387610334